In [153]:
import numpy as np
import pandas as pd
from datetime import date

import warnings
warnings.filterwarnings("ignore")

In [ ]:
"""
dataset split:
                      (date_received)                              
           dateset3: 20160701~20160731 (113640),features3 from 20160315~20160630  (off_test)
           dateset2: 20160515~20160615 (258446),features2 from 20160201~20160514  
           dateset1: 20160414~20160514 (138303),features1 from 20160101~20160413        
1.merchant related: 
      sales_use_coupon. total_coupon
      transfer_rate = sales_use_coupon/total_coupon.
      merchant_avg_distance,merchant_min_distance,merchant_max_distance of those use coupon 
      total_sales.  coupon_rate = sales_use_coupon/total_sales.  
       
2.coupon related: 
      discount_rate. discount_man. discount_jian. is_man_jian
      day_of_week,day_of_month. (date_received)
      
3.user related: 
      distance. 
      user_avg_distance, user_min_distance,user_max_distance. 
      buy_use_coupon. buy_total. coupon_received.
      buy_use_coupon/coupon_received. 
      avg_diff_date_datereceived. min_diff_date_datereceived. max_diff_date_datereceived.  
      count_merchant.  
4.user_merchant:
      times_user_buy_merchant_before.
     
5. other feature:
      this_month_user_receive_all_coupon_count
      this_month_user_receive_same_coupon_count
      this_month_user_receive_same_coupon_lastone
      this_month_user_receive_same_coupon_firstone
      this_day_user_receive_all_coupon_count
      this_day_user_receive_same_coupon_count
      day_gap_before, day_gap_after  (receive the same coupon)
"""

In [3]:
off_train = pd.read_csv('data/ccf_offline_stage1_train.csv')
on_train = pd.read_csv('data/ccf_online_stage1_train.csv')
off_test = pd.read_csv('data/ccf_offline_stage1_test_revised.csv')

In [15]:
off_test.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,4129537,450,9983,30:5,1,20160712
1,6949378,1300,3429,30:5,null,20160706
2,2166529,7113,6928,200:20,5,20160727
3,2166529,7113,1808,100:10,5,20160727
4,6172162,7605,6500,30:1,2,20160708


In [14]:
feature1 = off_train[(off_train['Date'] >= '20160101') & (off_train['Date'] <= '20160413') | (off_train['Date'] == 'null') & (off_train['Date_received'] >= '20160101') & (off_train['Date_received'] <= '20160413') ]
dataset1 = off_train[(off_train['Date_received'] >= '20160414') & (off_train['Date_received'] <= '20160514')]
feature2 = off_train[(off_train['Date'] >= '20160201') & (off_train['Date'] <= '20160514') | (off_train['Date'] == 'null') & (off_train['Date_received'] >= '20160201') & (off_train['Date_received'] <= '20160514') ]
dataset2 = off_train[(off_train['Date_received'] >= '20160515') & (off_train['Date_received'] <= '20160615')]
feature3 = off_train[(off_train['Date'] >= '20160315') & (off_train['Date'] <= '20160630') | (off_train['Date'] == 'null') & (off_train['Date_received'] >= '20160315') & (off_train['Date_received'] <= '20160630') ]
dataset3 = off_test

## 1. other features

In [157]:
# 计算在 dataset 3 中每位 user 领取了多少次 coupon

t = dataset3[['User_id']]
t['this_month_user_receive_all_coupon_count'] = 1
t = t.groupby('User_id').agg('sum').reset_index()

# 计算在 dataset3 中每位 user 领取了多少相同的 coupon

t1 = dataset3[['User_id','Coupon_id']]
t1['this_month_user_receive_same_coupon_count'] = 1
t1 = t1.groupby(['User_id','Coupon_id']).agg('sum').reset_index()
t1.head(1)

,User_id,Coupon_id,this_month_user_receive_same_coupon_count
0,209,825,1


In [158]:
t2 = dataset3[['User_id','Coupon_id','Date_received']]
t2.Date_received = t2.Date_received.astype('str')
t2 = t2.groupby(['User_id','Coupon_id'])['Date_received'].agg(lambda x:':'.join(x)).reset_index()
t2['receive_number'] = t2.Date_received.apply(lambda s:len(s.split(':')))
t2 = t2[t2.receive_number>1]
t2['max_date_received'] = t2.Date_received.apply(lambda s:max([int(d) for d in s.split(':')]))
t2['min_date_received'] = t2.Date_received.apply(lambda s:min([int(d) for d in s.split(':')]))
t2 = t2[['User_id','Coupon_id','max_date_received','min_date_received']]

In [54]:
t2.head(1)

,User_id,Coupon_id,max_date_received,min_date_received
6,448,10927,20160710,20160704


In [104]:
# user 是否在这个月内 第一次 领取同样的 coupon 多次 （是：firstone = 1；不是第一次了：firstone = 0）
# user 是否在这个月内 最后一次 领取同样的 coupon 多次 （是：lastone = 1；后面还会继续领取：lastone = 0）

t3 = dataset3[['User_id','Coupon_id','Date_received']]
t3 = pd.merge(t3,t2,on=['User_id','Coupon_id'],how='left')
t3['this_month_user_receive_same_coupon_lastone'] = t3.max_date_received - t3.Date_received
t3['this_month_user_receive_same_coupon_firstone'] = t3.Date_received - t3.min_date_received

def is_firstlastone(x):
    if x==0:
        return 1
    elif x>0:
        return 0
    else:
        return -1 #those only receive once
        
t3.this_month_user_receive_same_coupon_lastone = t3.this_month_user_receive_same_coupon_lastone.apply(is_firstlastone)
t3.this_month_user_receive_same_coupon_firstone = t3.this_month_user_receive_same_coupon_firstone.apply(is_firstlastone)
t3 = t3[['User_id','Coupon_id','Date_received','this_month_user_receive_same_coupon_lastone','this_month_user_receive_same_coupon_firstone']]
t3.head(1)

,User_id,Coupon_id,Date_received,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone
0,4129537,9983,20160712,-1,-1


In [159]:
# 在某一天 user 领取的 coupon 数量

t4 = dataset3[['User_id','Date_received']]
t4['this_day_user_receive_all_coupon_count'] = 1
t4 = t4.groupby(['User_id','Date_received']).agg('sum').reset_index()
t4.head(1)

,User_id,Date_received,this_day_user_receive_all_coupon_count
0,209,20160721,2


In [160]:
# 在 Date_received 当天 user 领取相同 coupon 的数量

t5 = dataset3[['User_id','Coupon_id','Date_received']]
t5['this_day_user_receive_same_coupon_count'] = 1
t5 = t5.groupby(['User_id','Coupon_id','Date_received']).agg('sum').reset_index()
t5.head(1)

,User_id,Coupon_id,Date_received,this_day_user_receive_same_coupon_count
0,209,825,20160721,1


In [161]:
# user 领取相同 coupon 的日期

t6 = dataset3[['User_id','Coupon_id','Date_received']]
t6.Date_received = t6.Date_received.astype('str')
t6 = t6.groupby(['User_id','Coupon_id'])['Date_received'].agg(lambda x:':'.join(x)).reset_index()
t6.rename(columns={'Date_received':'dates'},inplace=True)
t6.head(10)

,User_id,Coupon_id,dates
0,209,825,20160721
1,209,7557,20160721
2,215,5488,20160703
3,316,3992,20160721
4,417,12465,20160712
5,432,10438,20160706
6,448,10927,20160710:20160707:20160706:20160704
7,448,13602,20160701
8,452,3992,20160721
9,452,6500,20160702


In [128]:
def get_day_gap_before(s):
    date_received,dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        this_gap = (date(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8]))-date(int(d[0:4]),int(d[4:6]),int(d[6:8]))).days
        if this_gap>0:
            gaps.append(this_gap)
    if len(gaps)==0:
        return -1
    else:
        return min(gaps)
        
def get_day_gap_after(s):
    date_received,dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        this_gap = (date(int(d[0:4]),int(d[4:6]),int(d[6:8]))-date(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8]))).days
        if this_gap>0:
            gaps.append(this_gap)
    if len(gaps)==0:
        return -1
    else:
        return min(gaps)
    
    
t7 = dataset3[['User_id','Coupon_id','Date_received']]
t7 = pd.merge(t7,t6,on=['User_id','Coupon_id'],how='left')
t7['date_received_date'] = t7.Date_received.astype('str') + '-' + t7.dates
t7['day_gap_before'] = t7.date_received_date.apply(get_day_gap_before)
t7['day_gap_after'] = t7.date_received_date.apply(get_day_gap_after)
t7 = t7[['User_id','Coupon_id','Date_received','day_gap_before','day_gap_after']]
t7.head(1)

,User_id,Coupon_id,Date_received,day_gap_before,day_gap_after
0,4129537,9983,20160712,-1,-1


In [137]:
other_feature3 = pd.merge(t1,t,on='User_id')
other_feature3 = pd.merge(other_feature3,t3,on=['User_id','Coupon_id'])
other_feature3 = pd.merge(other_feature3,t4,on=['User_id','Date_received'])
other_feature3 = pd.merge(other_feature3,t5,on=['User_id','Coupon_id','Date_received'])
other_feature3 = pd.merge(other_feature3,t7,on=['User_id','Coupon_id','Date_received'])
other_feature3.to_csv('data/other_feature3.csv',index=None)
print(other_feature3.shape)
other_feature3.head()

(262240, 11)


,User_id,Coupon_id,this_month_user_receive_same_coupon_count,this_month_user_receive_all_coupon_count,Date_received,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone,this_day_user_receive_all_coupon_count,this_day_user_receive_same_coupon_count,day_gap_before,day_gap_after
0,4,2902,1,1,20160607,-1,-1,1,1,-1,-1
1,165,7571,1,1,20160525,-1,-1,1,1,-1,-1
2,166,9261,1,1,20160525,-1,-1,1,1,-1,-1
3,215,8944,1,1,20160524,-1,-1,1,1,-1,-1
4,236,11002,1,1,20160528,-1,-1,1,1,-1,-1


In [156]:
t = dataset2[['User_id']]
t['this_month_user_receive_all_coupon_count'] = 1
t = t.groupby('User_id').agg('sum').reset_index()

t1 = dataset2[['User_id','Coupon_id']]
t1['this_month_user_receive_same_coupon_count'] = 1
t1 = t1.groupby(['User_id','Coupon_id']).agg('sum').reset_index()

t2 = dataset2[['User_id','Coupon_id','Date_received']]
t2.Date_received = t2.Date_received.astype('str')
t2 = t2.groupby(['User_id','Coupon_id'])['Date_received'].agg(lambda x:':'.join(x)).reset_index()
t2['receive_number'] = t2.Date_received.apply(lambda s:len(s.split(':')))
t2 = t2[t2.receive_number>1]
t2['max_date_received'] = t2.Date_received.apply(lambda s:max([int(d) for d in s.split(':')]))
t2['min_date_received'] = t2.Date_received.apply(lambda s:min([int(d) for d in s.split(':')]))
t2 = t2[['User_id','Coupon_id','max_date_received','min_date_received']]

t3 = dataset2[['User_id','Coupon_id','Date_received']]
t3 = pd.merge(t3,t2,on=['User_id','Coupon_id'],how='left')
t3['this_month_user_receive_same_coupon_lastone'] = t3.max_date_received - t3.Date_received.astype('int')
t3['this_month_user_receive_same_coupon_firstone'] = t3.Date_received.astype('int') - t3.min_date_received

def is_firstlastone(x):
    if x==0:
        return 1
    elif x>0:
        return 0
    else:
        return -1 #those only receive once
        
t3.this_month_user_receive_same_coupon_lastone = t3.this_month_user_receive_same_coupon_lastone.apply(is_firstlastone)
t3.this_month_user_receive_same_coupon_firstone = t3.this_month_user_receive_same_coupon_firstone.apply(is_firstlastone)
t3 = t3[['User_id','Coupon_id','Date_received','this_month_user_receive_same_coupon_lastone','this_month_user_receive_same_coupon_firstone']]

t4 = dataset2[['User_id','Date_received']]
t4['this_day_user_receive_all_coupon_count'] = 1
t4 = t4.groupby(['User_id','Date_received']).agg('sum').reset_index()

t5 = dataset2[['User_id','Coupon_id','Date_received']]
t5['this_day_user_receive_same_coupon_count'] = 1
t5 = t5.groupby(['User_id','Coupon_id','Date_received']).agg('sum').reset_index()

t6 = dataset2[['User_id','Coupon_id','Date_received']]
t6.Date_received = t6.Date_received.astype('str')
t6 = t6.groupby(['User_id','Coupon_id'])['Date_received'].agg(lambda x:':'.join(x)).reset_index()
t6.rename(columns={'Date_received':'dates'},inplace=True)

def get_day_gap_before(s):
    date_received,dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        this_gap = (date(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8]))-date(int(d[0:4]),int(d[4:6]),int(d[6:8]))).days
        if this_gap>0:
            gaps.append(this_gap)
    if len(gaps)==0:
        return -1
    else:
        return min(gaps)
        
def get_day_gap_after(s):
    date_received,dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        this_gap = (date(int(d[0:4]),int(d[4:6]),int(d[6:8]))-date(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8]))).days
        if this_gap>0:
            gaps.append(this_gap)
    if len(gaps)==0:
        return -1
    else:
        return min(gaps)
    

t7 = dataset2[['User_id','Coupon_id','Date_received']]
t7 = pd.merge(t7,t6,on=['User_id','Coupon_id'],how='left')
t7['date_received_date'] = t7.Date_received.astype('str') + '-' + t7.dates
t7['day_gap_before'] = t7.date_received_date.apply(get_day_gap_before)
t7['day_gap_after'] = t7.date_received_date.apply(get_day_gap_after)
t7 = t7[['User_id','Coupon_id','Date_received','day_gap_before','day_gap_after']]

other_feature2 = pd.merge(t1,t,on='User_id')
other_feature2 = pd.merge(other_feature2,t3,on=['User_id','Coupon_id'])
other_feature2 = pd.merge(other_feature2,t4,on=['User_id','Date_received'])
other_feature2 = pd.merge(other_feature2,t5,on=['User_id','Coupon_id','Date_received'])
other_feature2 = pd.merge(other_feature2,t7,on=['User_id','Coupon_id','Date_received'])
other_feature2.to_csv('data/other_feature2.csv',index=None)
print (other_feature2.shape)
other_feature2.head()

(262240, 11)


,User_id,Coupon_id,this_month_user_receive_same_coupon_count,this_month_user_receive_all_coupon_count,Date_received,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone,this_day_user_receive_all_coupon_count,this_day_user_receive_same_coupon_count,day_gap_before,day_gap_after
0,4,2902,1,1,20160607,-1,-1,1,1,-1,-1
1,165,7571,1,1,20160525,-1,-1,1,1,-1,-1
2,166,9261,1,1,20160525,-1,-1,1,1,-1,-1
3,215,8944,1,1,20160524,-1,-1,1,1,-1,-1
4,236,11002,1,1,20160528,-1,-1,1,1,-1,-1


In [155]:
#for dataset1
t = dataset1[['User_id']]
t['this_month_user_receive_all_coupon_count'] = 1
t = t.groupby('User_id').agg('sum').reset_index()

t1 = dataset1[['User_id','Coupon_id']]
t1['this_month_user_receive_same_coupon_count'] = 1
t1 = t1.groupby(['User_id','Coupon_id']).agg('sum').reset_index()

t2 = dataset1[['User_id','Coupon_id','Date_received']]
t2.Date_received = t2.Date_received.astype('str')
t2 = t2.groupby(['User_id','Coupon_id'])['Date_received'].agg(lambda x:':'.join(x)).reset_index()
t2['receive_number'] = t2.Date_received.apply(lambda s:len(s.split(':')))
t2 = t2[t2.receive_number>1]
t2['max_date_received'] = t2.Date_received.apply(lambda s:max([int(d) for d in s.split(':')]))
t2['min_date_received'] = t2.Date_received.apply(lambda s:min([int(d) for d in s.split(':')]))
t2 = t2[['User_id','Coupon_id','max_date_received','min_date_received']]

t3 = dataset1[['User_id','Coupon_id','Date_received']]
t3 = pd.merge(t3,t2,on=['User_id','Coupon_id'],how='left')
t3['this_month_user_receive_same_coupon_lastone'] = t3.max_date_received - t3.Date_received.astype('int')
t3['this_month_user_receive_same_coupon_firstone'] = t3.Date_received.astype('int') - t3.min_date_received

def is_firstlastone(x):
    if x==0:
        return 1
    elif x>0:
        return 0
    else:
        return -1 #those only receive once
        
t3.this_month_user_receive_same_coupon_lastone = t3.this_month_user_receive_same_coupon_lastone.apply(is_firstlastone)
t3.this_month_user_receive_same_coupon_firstone = t3.this_month_user_receive_same_coupon_firstone.apply(is_firstlastone)
t3 = t3[['User_id','Coupon_id','Date_received','this_month_user_receive_same_coupon_lastone','this_month_user_receive_same_coupon_firstone']]

t4 = dataset1[['User_id','Date_received']]
t4['this_day_user_receive_all_coupon_count'] = 1
t4 = t4.groupby(['User_id','Date_received']).agg('sum').reset_index()

t5 = dataset1[['User_id','Coupon_id','Date_received']]
t5['this_day_user_receive_same_coupon_count'] = 1
t5 = t5.groupby(['User_id','Coupon_id','Date_received']).agg('sum').reset_index()

t6 = dataset1[['User_id','Coupon_id','Date_received']]
t6.Date_received = t6.Date_received.astype('str')
t6 = t6.groupby(['User_id','Coupon_id'])['Date_received'].agg(lambda x:':'.join(x)).reset_index()
t6.rename(columns={'Date_received':'dates'},inplace=True)

def get_day_gap_before(s):
    date_received,dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        this_gap = (date(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8]))-date(int(d[0:4]),int(d[4:6]),int(d[6:8]))).days
        if this_gap>0:
            gaps.append(this_gap)
    if len(gaps)==0:
        return -1
    else:
        return min(gaps)
        
def get_day_gap_after(s):
    date_received,dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        this_gap = (date(int(d[0:4]),int(d[4:6]),int(d[6:8]))-date(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8]))).days
        if this_gap>0:
            gaps.append(this_gap)
    if len(gaps)==0:
        return -1
    else:
        return min(gaps)
    

t7 = dataset1[['User_id','Coupon_id','Date_received']]
t7 = pd.merge(t7,t6,on=['User_id','Coupon_id'],how='left')
t7['date_received_date'] = t7.Date_received.astype('str') + '-' + t7.dates
t7['day_gap_before'] = t7.date_received_date.apply(get_day_gap_before)
t7['day_gap_after'] = t7.date_received_date.apply(get_day_gap_after)
t7 = t7[['User_id','Coupon_id','Date_received','day_gap_before','day_gap_after']]

other_feature1 = pd.merge(t1,t,on='User_id')
other_feature1 = pd.merge(other_feature1,t3,on=['User_id','Coupon_id'])
other_feature1 = pd.merge(other_feature1,t4,on=['User_id','Date_received'])
other_feature1 = pd.merge(other_feature1,t5,on=['User_id','Coupon_id','Date_received'])
other_feature1 = pd.merge(other_feature1,t7,on=['User_id','Coupon_id','Date_received'])
other_feature1.to_csv('data/other_feature1.csv',index=None)
print (other_feature1.shape)
other_feature1.head()

(139785, 11)


,User_id,Coupon_id,this_month_user_receive_same_coupon_count,this_month_user_receive_all_coupon_count,Date_received,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone,this_day_user_receive_all_coupon_count,this_day_user_receive_same_coupon_count,day_gap_before,day_gap_after
0,173,7610,1,1,20160414,-1,-1,1,1,-1,-1
1,285,1532,1,1,20160501,-1,-1,1,1,-1,-1
2,316,8952,1,1,20160430,-1,-1,1,1,-1,-1
3,377,2857,1,1,20160512,-1,-1,1,1,-1,-1
4,387,7610,1,1,20160421,-1,-1,1,1,-1,-1


## 2. coupon-related feature

In [ ]:
"""
2.coupon related: 
      discount_rate. discount_man. discount_jian. is_man_jian
      day_of_week,day_of_month. (date_received)
"""

In [142]:
def calc_discount_rate(s):
    s =str(s)
    s = s.split(':')
    if len(s)==1:
        return float(s[0])
    else:
        return 1.0-float(s[1])/float(s[0])

def get_discount_man(s):
    s =str(s)
    s = s.split(':')
    if len(s)==1:
        return 'null'
    else:
        return int(s[0])
        
def get_discount_jian(s):
    s =str(s)
    s = s.split(':')
    if len(s)==1:
        return 'null'
    else:
        return int(s[1])

def is_man_jian(s):
    s =str(s)
    s = s.split(':')
    if len(s)==1:
        return 0
    else:
        return 1

In [143]:
#dataset3
dataset3['day_of_week'] = dataset3.Date_received.astype('str').apply(lambda x:date(int(x[0:4]),int(x[4:6]),int(x[6:8])).weekday()+1)
dataset3['day_of_month'] = dataset3.Date_received.astype('str').apply(lambda x:int(x[6:8]))
dataset3['days_distance'] = dataset3.Date_received.astype('str').apply(lambda x:(date(int(x[0:4]),int(x[4:6]),int(x[6:8]))-date(2016,6,30)).days)
dataset3['discount_man'] = dataset3.Discount_rate.apply(get_discount_man)
dataset3['discount_jian'] = dataset3.Discount_rate.apply(get_discount_jian)
dataset3['is_man_jian'] = dataset3.Discount_rate.apply(is_man_jian)
dataset3['discount_rate'] = dataset3.Discount_rate.apply(calc_discount_rate)

In [151]:
d = dataset3[['Coupon_id']]
d['coupon_count'] = 1
d = d.groupby('Coupon_id').agg('sum').reset_index()

dataset3 = pd.merge(dataset3,d,on='Coupon_id',how='left')
dataset3.to_csv('data/coupon3_feature.csv',index=None)

dataset3.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,day_of_week,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,discount_rate,coupon_count
0,4129537,450,9983,30:5,1,20160712,2,12,12,30,5,1,0.833333,11586
1,6949378,1300,3429,30:5,null,20160706,3,6,6,30,5,1,0.833333,4887
2,2166529,7113,6928,200:20,5,20160727,3,27,27,200,20,1,0.900000,205
3,2166529,7113,1808,100:10,5,20160727,3,27,27,100,10,1,0.900000,205
4,6172162,7605,6500,30:1,2,20160708,5,8,8,30,1,1,0.966667,132


In [154]:
#dataset2
dataset2['day_of_week'] = dataset2.Date_received.astype('str').apply(lambda x:date(int(x[0:4]),int(x[4:6]),int(x[6:8])).weekday()+1)
dataset2['day_of_month'] = dataset2.Date_received.astype('str').apply(lambda x:int(x[6:8]))
dataset2['days_distance'] = dataset2.Date_received.astype('str').apply(lambda x:(date(int(x[0:4]),int(x[4:6]),int(x[6:8]))-date(2016,5,14)).days)
dataset2['discount_man'] = dataset2.Discount_rate.apply(get_discount_man)
dataset2['discount_jian'] = dataset2.Discount_rate.apply(get_discount_jian)
dataset2['is_man_jian'] = dataset2.Discount_rate.apply(is_man_jian)
dataset2['discount_rate'] = dataset2.Discount_rate.apply(calc_discount_rate)
d = dataset2[['Coupon_id']]
d['coupon_count'] = 1
d = d.groupby('Coupon_id').agg('sum').reset_index()

dataset2 = pd.merge(dataset2,d,on='Coupon_id',how='left')
dataset2.to_csv('data/coupon2_feature.csv',index=None)

dataset2.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,day_of_week,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,discount_rate,coupon_count_x,coupon_count_y
0,1439408,4663,11002,150:20,1,20160528,null,6,28,14,150,20,1,0.866667,7730,7730
1,1439408,2632,8591,20:1,0,20160613,null,1,13,30,20,1,1,0.950000,5,5
2,1439408,2632,8591,20:1,0,20160516,20160613,1,16,2,20,1,1,0.950000,5,5
3,2029232,450,1532,30:5,0,20160530,null,1,30,16,30,5,1,0.833333,11728,11728
4,2029232,6459,12737,20:1,0,20160519,null,4,19,5,20,1,1,0.950000,16,16


In [162]:
#dataset1
dataset1['day_of_week'] = dataset1.Date_received.astype('str').apply(lambda x:date(int(x[0:4]),int(x[4:6]),int(x[6:8])).weekday()+1)
dataset1['day_of_month'] = dataset1.Date_received.astype('str').apply(lambda x:int(x[6:8]))
dataset1['days_distance'] = dataset1.Date_received.astype('str').apply(lambda x:(date(int(x[0:4]),int(x[4:6]),int(x[6:8]))-date(2016,4,13)).days)
dataset1['discount_man'] = dataset1.Discount_rate.apply(get_discount_man)
dataset1['discount_jian'] = dataset1.Discount_rate.apply(get_discount_jian)
dataset1['is_man_jian'] = dataset1.Discount_rate.apply(is_man_jian)
dataset1['discount_rate'] = dataset1.Discount_rate.apply(calc_discount_rate)
d = dataset1[['Coupon_id']]
d['coupon_count'] = 1
d = d.groupby('Coupon_id').agg('sum').reset_index()

dataset1 = pd.merge(dataset1,d,on='Coupon_id',how='left')
dataset1.to_csv('data/coupon1_feature.csv',index=None)

dataset1.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,day_of_week,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,discount_rate,coupon_count
0,1832624,3381,7610,200:20,0,20160429,null,5,29,16,200,20,1,0.900000,22281
1,163606,1569,5054,200:30,10,20160421,null,4,21,8,200,30,1,0.850000,11893
2,4061024,3381,7610,200:20,10,20160426,null,2,26,13,200,20,1,0.900000,22281
3,106443,450,3732,30:5,null,20160429,null,5,29,16,30,5,1,0.833333,8845
4,114747,1569,5054,200:30,9,20160426,null,2,26,13,200,30,1,0.850000,11893


## 3. merchant_related features

In [ ]:
"""
1.merchant related: 
      total_sales. sales_use_coupon.  total_coupon
      coupon_rate = sales_use_coupon/total_sales.  
      transfer_rate = sales_use_coupon/total_coupon. 
      merchant_avg_distance,merchant_min_distance,merchant_max_distance of those use coupon
"""

In [196]:
#for dataset3
merchant3 = feature3[['Merchant_id','Coupon_id','Distance','Date_received','Date']]

t = merchant3[['Merchant_id']]
t.drop_duplicates(inplace=True)                          # drop_duplicates 去除重复项

# 每个商品的销售次数

t1 = merchant3[merchant3.Date !='null'][['Merchant_id']]
t1['total_sales'] = 1
t1 = t1.groupby('Merchant_id').agg('sum').reset_index()

# 消费时使用 coupon 的次数

t2 = merchant3[(merchant3.Date!='null')&(merchant3.Coupon_id!='null')][['Merchant_id']]
t2['sales_use_coupon'] = 1
t2 = t2.groupby('Merchant_id').agg('sum').reset_index()

# 在商品上使用的 coupon 总数

t3 = merchant3[merchant3.Coupon_id!='null'][['Merchant_id']]
t3['total_coupon'] = 1
t3 = t3.groupby('Merchant_id').agg('sum').reset_index()

# 使用 coupon 购买商品时的 distance

t4 = merchant3[(merchant3.Date!='null')&(merchant3.Coupon_id!='null')][['Merchant_id','Distance']]
t4.replace('null',-1,inplace=True)
t4.Distance = t4.Distance.astype('int')
t4.replace(-1,np.nan,inplace=True)

# 使用 coupon 购买商品时的 distance 最小值

t5 = t4.groupby('Merchant_id').agg('min').reset_index()
t5.rename(columns={'Distance':'merchant_min_distance'},inplace=True)

# 使用 coupon 购买商品时的 distance 最大值

t6 = t4.groupby('Merchant_id').agg('max').reset_index()
t6.rename(columns={'Distance':'merchant_max_distance'},inplace=True)

# 使用 coupon 购买商品时的 distance 平均值

t7 = t4.groupby('Merchant_id').agg('mean').reset_index()
t7.rename(columns={'Distance':'merchant_mean_distance'},inplace=True)

# 使用 coupon 购买商品时的 distance 中位数值

t8 = t4.groupby('Merchant_id').agg('median').reset_index()
t8.rename(columns={'Distance':'merchant_median_distance'},inplace=True)

merchant2_feature = pd.merge(t,t1,on='Merchant_id',how='left')
merchant2_feature = pd.merge(merchant2_feature,t2,on='Merchant_id',how='left')
merchant2_feature = pd.merge(merchant2_feature,t3,on='Merchant_id',how='left')
merchant2_feature = pd.merge(merchant2_feature,t5,on='Merchant_id',how='left')
merchant2_feature = pd.merge(merchant2_feature,t6,on='Merchant_id',how='left')
merchant2_feature = pd.merge(merchant2_feature,t7,on='Merchant_id',how='left')
merchant2_feature = pd.merge(merchant2_feature,t8,on='Merchant_id',how='left')
merchant2_feature.sales_use_coupon = merchant2_feature.sales_use_coupon.replace(np.nan,0) #fillna with 0
merchant2_feature['merchant_coupon_transfer_rate'] = merchant2_feature.sales_use_coupon.astype('float') / merchant2_feature.total_coupon
merchant2_feature['coupon_rate'] = merchant2_feature.sales_use_coupon.astype('float') / merchant2_feature.total_sales
merchant2_feature.total_coupon = merchant2_feature.total_coupon.replace(np.nan,0) #fillna with 0
merchant2_feature.to_csv('data/merchant2_feature.csv',index=None)
merchant2_feature.head()

,Merchant_id,total_sales,sales_use_coupon,total_coupon,merchant_min_distance,merchant_max_distance,merchant_mean_distance,merchant_median_distance,merchant_coupon_transfer_rate,coupon_rate
0,4663,393.0,39.0,11445.0,0.0,10.0,1.911765,1.0,0.003408,0.099237
1,2632,8.0,3.0,23.0,0.0,1.0,0.666667,1.0,0.130435,0.375000
2,3381,9579.0,416.0,46729.0,0.0,10.0,1.713580,1.0,0.008902,0.043428
3,450,7757.0,1239.0,35933.0,0.0,10.0,0.894068,0.0,0.034481,0.159727
4,6459,20.0,0.0,16.0,NaN,NaN,NaN,NaN,0.000000,0.000000


In [198]:
#for dataset1
merchant1 = feature1[['Merchant_id','Coupon_id','Distance','Date_received','Date']]

t = merchant1[['Merchant_id']]
t.drop_duplicates(inplace=True)

t1 = merchant1[merchant1.Date!='null'][['Merchant_id']]
t1['total_sales'] = 1
t1 = t1.groupby('Merchant_id').agg('sum').reset_index()

t2 = merchant1[(merchant1.Date!='null')&(merchant1.Coupon_id!='null')][['Merchant_id']]
t2['sales_use_coupon'] = 1
t2 = t2.groupby('Merchant_id').agg('sum').reset_index()

t3 = merchant1[merchant1.Coupon_id!='null'][['Merchant_id']]
t3['total_coupon'] = 1
t3 = t3.groupby('Merchant_id').agg('sum').reset_index()

t4 = merchant1[(merchant1.Date!='null')&(merchant1.Coupon_id!='null')][['Merchant_id','Distance']]
t4.replace('null',-1,inplace=True)
t4.Distance = t4.Distance.astype('int')
t4.replace(-1,np.nan,inplace=True)
t5 = t4.groupby('Merchant_id').agg('min').reset_index()
t5.rename(columns={'Distance':'merchant_min_distance'},inplace=True)

t6 = t4.groupby('Merchant_id').agg('max').reset_index()
t6.rename(columns={'Distance':'merchant_max_distance'},inplace=True)

t7 = t4.groupby('Merchant_id').agg('mean').reset_index()
t7.rename(columns={'Distance':'merchant_mean_distance'},inplace=True)

t8 = t4.groupby('Merchant_id').agg('median').reset_index()
t8.rename(columns={'Distance':'merchant_median_distance'},inplace=True)


merchant1_feature = pd.merge(t,t1,on='Merchant_id',how='left')
merchant1_feature = pd.merge(merchant1_feature,t2,on='Merchant_id',how='left')
merchant1_feature = pd.merge(merchant1_feature,t3,on='Merchant_id',how='left')
merchant1_feature = pd.merge(merchant1_feature,t5,on='Merchant_id',how='left')
merchant1_feature = pd.merge(merchant1_feature,t6,on='Merchant_id',how='left')
merchant1_feature = pd.merge(merchant1_feature,t7,on='Merchant_id',how='left')
merchant1_feature = pd.merge(merchant1_feature,t8,on='Merchant_id',how='left')
merchant1_feature.sales_use_coupon = merchant1_feature.sales_use_coupon.replace(np.nan,0) #fillna with 0
merchant1_feature['merchant_coupon_transfer_rate'] = merchant1_feature.sales_use_coupon.astype('float') / merchant1_feature.total_coupon
merchant1_feature['coupon_rate'] = merchant1_feature.sales_use_coupon.astype('float') / merchant1_feature.total_sales
merchant1_feature.total_coupon = merchant1_feature.total_coupon.replace(np.nan,0) #fillna with 0
merchant1_feature.to_csv('data/merchant1_feature.csv',index=None)
merchant1_feature.head()

,Merchant_id,total_sales,sales_use_coupon,total_coupon,merchant_min_distance,merchant_max_distance,merchant_mean_distance,merchant_median_distance,merchant_coupon_transfer_rate,coupon_rate
0,2632,14.0,1.0,28.0,1.0,1.0,1.000000,1.0,0.035714,0.071429
1,3381,14962.0,2168.0,100369.0,0.0,10.0,1.650457,1.0,0.021600,0.144900
2,2099,5783.0,1705.0,16824.0,0.0,10.0,0.968072,0.0,0.101343,0.294830
3,4833,522.0,116.0,8321.0,0.0,10.0,3.037736,2.0,0.013941,0.222222
4,8390,1056.0,133.0,690.0,0.0,10.0,0.865079,0.0,0.192754,0.125947


## 4. user-related features

In [ ]:
"""
3.user related: 
      count_merchant. 
      user_avg_distance, user_min_distance,user_max_distance. 
      buy_use_coupon. buy_total. coupon_received.
      buy_use_coupon/coupon_received. 
      buy_use_coupon/buy_total
      user_date_datereceived_gap
      
"""

In [199]:
def get_user_date_datereceived_gap(s):
    s = s.split(':')
    return (date(int(s[0][0:4]),int(s[0][4:6]),int(s[0][6:8])) - date(int(s[1][0:4]),int(s[1][4:6]),int(s[1][6:8]))).days

In [233]:
#for dataset3
user3 = feature3[['User_id','Merchant_id','Coupon_id','Discount_rate','Distance','Date_received','Date']]

t = user3[['User_id']]
t.drop_duplicates(inplace=True)

# 每个 user 使用 coupon 购买商品的次数

t1 = user3[user3.Date!='null'][['User_id','Merchant_id']]
t1.drop_duplicates(inplace=True)
t1.Merchant_id = 1
t1 = t1.groupby('User_id').agg('sum').reset_index()
t1.rename(columns={'Merchant_id':'count_merchant'},inplace=True)

# 每个 user 使用 coupon 购买商品的 distance

t2 = user3[(user3.Date!='null')&(user3.Coupon_id!='null')][['User_id','Distance']]
t2.replace('null',-1,inplace=True)
t2.Distance = t2.Distance.astype('int')
t2.replace(-1,np.nan,inplace=True)

# 每个 user 使用 coupon 购买商品的 distance 的最大值，最小值，平均值和中位数

t3 = t2.groupby('User_id').agg('min').reset_index()
t3.rename(columns={'Distance':'user_min_distance'},inplace=True)

t4 = t2.groupby('User_id').agg('max').reset_index()
t4.rename(columns={'Distance':'user_max_distance'},inplace=True)

t5 = t2.groupby('User_id').agg('mean').reset_index()
t5.rename(columns={'Distance':'user_mean_distance'},inplace=True)

t6 = t2.groupby('User_id').agg('median').reset_index()
t6.rename(columns={'Distance':'user_median_distance'},inplace=True)

# 每个 user 使用 coupon 购买商品的次数

t7 = user3[(user3.Date!='null')&(user3.Coupon_id!='null')][['User_id']]
t7['buy_use_coupon'] = 1
t7 = t7.groupby('User_id').agg('sum').reset_index()

# 每个 user 购买商品的次数

t8 = user3[user3.Date!='null'][['User_id']]
t8['buy_total'] = 1
t8 = t8.groupby('User_id').agg('sum').reset_index()

# 每个 user 领取的 coupon 数量

t9 = user3[user3.Coupon_id!='null'][['User_id']]
t9['coupon_received'] = 1
t9 = t9.groupby('User_id').agg('sum').reset_index()

# date - date_received

t10 = user3[(user3.Date_received!='null')&(user3.Date!='null')][['User_id','Date_received','Date']]
t10['user_date_datereceived_gap'] = t10.Date + ':' + t10.Date_received
t10.user_date_datereceived_gap = t10.user_date_datereceived_gap.apply(get_user_date_datereceived_gap)
t10 = t10[['User_id','user_date_datereceived_gap']]

# 每个 user 的平均间隔使用时间

t11 = t10.groupby('User_id').agg('mean').reset_index()
t11.rename(columns={'User_date_datereceived_gap':'avg_user_date_datereceived_gap'},inplace=True)

t12 = t10.groupby('User_id').agg('min').reset_index()
t12.rename(columns={'user_date_datereceived_gap':'min_user_date_datereceived_gap'},inplace=True)

t13 = t10.groupby('User_id').agg('max').reset_index()
t13.rename(columns={'user_date_datereceived_gap':'max_user_date_datereceived_gap'},inplace=True)

user3_feature = pd.merge(t,t1,on='User_id',how='left')
user3_feature = pd.merge(user3_feature,t3,on='User_id',how='left')
user3_feature = pd.merge(user3_feature,t4,on='User_id',how='left')
user3_feature = pd.merge(user3_feature,t5,on='User_id',how='left')
user3_feature = pd.merge(user3_feature,t6,on='User_id',how='left')
user3_feature = pd.merge(user3_feature,t7,on='User_id',how='left')
user3_feature = pd.merge(user3_feature,t8,on='User_id',how='left')
user3_feature = pd.merge(user3_feature,t9,on='User_id',how='left')
user3_feature = pd.merge(user3_feature,t11,on='User_id',how='left')
user3_feature = pd.merge(user3_feature,t12,on='User_id',how='left')
user3_feature = pd.merge(user3_feature,t13,on='User_id',how='left')
user3_feature.count_merchant = user3_feature.count_merchant.replace(np.nan,0)
user3_feature.buy_use_coupon = user3_feature.buy_use_coupon.replace(np.nan,0)
user3_feature['buy_use_coupon_rate'] = user3_feature.buy_use_coupon.astype('float') / user3_feature.buy_total.astype('float')
user3_feature['user_coupon_transfer_rate'] = user3_feature.buy_use_coupon.astype('float') / user3_feature.coupon_received.astype('float')
user3_feature.buy_total = user3_feature.buy_total.replace(np.nan,0)
user3_feature.coupon_received = user3_feature.coupon_received.replace(np.nan,0)
user3_feature.to_csv('data/user3_feature.csv',index=None)
user3_feature.head()

,User_id,count_merchant,user_min_distance,user_max_distance,user_mean_distance,user_median_distance,buy_use_coupon,buy_total,coupon_received,user_date_datereceived_gap,min_user_date_datereceived_gap,max_user_date_datereceived_gap,buy_use_coupon_rate,user_coupon_transfer_rate
0,1439408,1.0,0.0,0.0,0.0,0.0,1.0,2.0,4.0,28.0,28.0,28.0,0.5,0.25
1,1832624,0.0,NaN,NaN,NaN,NaN,0.0,0.0,1.0,NaN,NaN,NaN,NaN,0.00
2,2029232,1.0,NaN,NaN,NaN,NaN,0.0,2.0,2.0,NaN,NaN,NaN,0.0,0.00
3,2747744,0.0,NaN,NaN,NaN,NaN,0.0,0.0,1.0,NaN,NaN,NaN,NaN,0.00
4,196342,1.0,NaN,NaN,NaN,NaN,0.0,1.0,1.0,NaN,NaN,NaN,0.0,0.00


In [235]:
#for dataset2
user2 = feature2[['User_id','Merchant_id','Coupon_id','Discount_rate','Distance','Date_received','Date']]

t = user2[['User_id']]
t.drop_duplicates(inplace=True)

t1 = user2[user2.Date!='null'][['User_id','Merchant_id']]
t1.drop_duplicates(inplace=True)
t1.Merchant_id = 1
t1 = t1.groupby('User_id').agg('sum').reset_index()
t1.rename(columns={'Merchant_id':'count_merchant'},inplace=True)

t2 = user2[(user2.Date!='null')&(user2.Coupon_id!='null')][['User_id','Distance']]
t2.replace('null',-1,inplace=True)
t2.Distance = t2.Distance.astype('int')
t2.replace(-1,np.nan,inplace=True)
t3 = t2.groupby('User_id').agg('min').reset_index()
t3.rename(columns={'Distance':'user_min_distance'},inplace=True)

t4 = t2.groupby('User_id').agg('max').reset_index()
t4.rename(columns={'Distance':'user_max_distance'},inplace=True)

t5 = t2.groupby('User_id').agg('mean').reset_index()
t5.rename(columns={'Distance':'user_mean_distance'},inplace=True)

t6 = t2.groupby('User_id').agg('median').reset_index()
t6.rename(columns={'Distance':'user_median_distance'},inplace=True)

t7 = user2[(user2.Date!='null')&(user2.Coupon_id!='null')][['User_id']]
t7['buy_use_coupon'] = 1
t7 = t7.groupby('User_id').agg('sum').reset_index()

t8 = user2[user2.Date!='null'][['User_id']]
t8['buy_total'] = 1
t8 = t8.groupby('User_id').agg('sum').reset_index()

t9 = user2[user2.Coupon_id!='null'][['User_id']]
t9['coupon_received'] = 1
t9 = t9.groupby('User_id').agg('sum').reset_index()

t10 = user2[(user2.Date_received!='null')&(user2.Date!='null')][['User_id','Date_received','Date']]
t10['user_date_datereceived_gap'] = t10.Date + ':' + t10.Date_received
t10.user_date_datereceived_gap = t10.user_date_datereceived_gap.apply(get_user_date_datereceived_gap)
t10 = t10[['User_id','user_date_datereceived_gap']]

t11 = t10.groupby('User_id').agg('mean').reset_index()
t11.rename(columns={'user_date_datereceived_gap':'avg_user_date_datereceived_gap'},inplace=True)
t12 = t10.groupby('User_id').agg('min').reset_index()
t12.rename(columns={'user_date_datereceived_gap':'min_user_date_datereceived_gap'},inplace=True)
t13 = t10.groupby('User_id').agg('max').reset_index()
t13.rename(columns={'user_date_datereceived_gap':'max_user_date_datereceived_gap'},inplace=True)

user2_feature = pd.merge(t,t1,on='User_id',how='left')
user2_feature = pd.merge(user2_feature,t3,on='User_id',how='left')
user2_feature = pd.merge(user2_feature,t4,on='User_id',how='left')
user2_feature = pd.merge(user2_feature,t5,on='User_id',how='left')
user2_feature = pd.merge(user2_feature,t6,on='User_id',how='left')
user2_feature = pd.merge(user2_feature,t7,on='User_id',how='left')
user2_feature = pd.merge(user2_feature,t8,on='User_id',how='left')
user2_feature = pd.merge(user2_feature,t9,on='User_id',how='left')
user2_feature = pd.merge(user2_feature,t11,on='User_id',how='left')
user2_feature = pd.merge(user2_feature,t12,on='User_id',how='left')
user2_feature = pd.merge(user2_feature,t13,on='User_id',how='left')
user2_feature.count_merchant = user2_feature.count_merchant.replace(np.nan,0)
user2_feature.buy_use_coupon = user2_feature.buy_use_coupon.replace(np.nan,0)
user2_feature['buy_use_coupon_rate'] = user2_feature.buy_use_coupon.astype('float') / user2_feature.buy_total.astype('float')
user2_feature['user_coupon_transfer_rate'] = user2_feature.buy_use_coupon.astype('float') / user2_feature.coupon_received.astype('float')
user2_feature.buy_total = user2_feature.buy_total.replace(np.nan,0)
user2_feature.coupon_received = user2_feature.coupon_received.replace(np.nan,0)
user2_feature.to_csv('data/user2_feature.csv',index=None)
user2_feature.head()

,User_id,count_merchant,user_min_distance,user_max_distance,user_mean_distance,user_median_distance,buy_use_coupon,buy_total,coupon_received,avg_user_date_datereceived_gap,min_user_date_datereceived_gap,max_user_date_datereceived_gap,buy_use_coupon_rate,user_coupon_transfer_rate
0,1439408,1.0,NaN,NaN,NaN,NaN,0.0,1.0,2.0,NaN,NaN,NaN,0.0,0.0
1,1832624,0.0,NaN,NaN,NaN,NaN,0.0,0.0,1.0,NaN,NaN,NaN,NaN,0.0
2,73611,0.0,NaN,NaN,NaN,NaN,0.0,0.0,1.0,NaN,NaN,NaN,NaN,0.0
3,163606,0.0,NaN,NaN,NaN,NaN,0.0,0.0,1.0,NaN,NaN,NaN,NaN,0.0
4,94107,0.0,NaN,NaN,NaN,NaN,0.0,0.0,1.0,NaN,NaN,NaN,NaN,0.0


In [238]:
#for dataset1
user1 = feature1[['User_id','Merchant_id','Coupon_id','Discount_rate','Distance','Date_received','Date']]

t = user1[['User_id']]
t.drop_duplicates(inplace=True)

t1 = user1[user1.Date!='null'][['User_id','Merchant_id']]
t1.drop_duplicates(inplace=True)
t1.Merchant_id = 1
t1 = t1.groupby('User_id').agg('sum').reset_index()
t1.rename(columns={'Merchant_id':'count_merchant'},inplace=True)

t2 = user1[(user1.Date!='null')&(user1.Coupon_id!='null')][['User_id','Distance']]
t2.replace('null',-1,inplace=True)
t2.Distance = t2.Distance.astype('int')
t2.replace(-1,np.nan,inplace=True)
t3 = t2.groupby('User_id').agg('min').reset_index()
t3.rename(columns={'Distance':'user_min_distance'},inplace=True)

t4 = t2.groupby('User_id').agg('max').reset_index()
t4.rename(columns={'Distance':'user_max_distance'},inplace=True)

t5 = t2.groupby('User_id').agg('mean').reset_index()
t5.rename(columns={'Distance':'user_mean_distance'},inplace=True)

t6 = t2.groupby('User_id').agg('median').reset_index()
t6.rename(columns={'Distance':'user_median_distance'},inplace=True)

t7 = user1[(user1.Date!='null')&(user1.Coupon_id!='null')][['User_id']]
t7['buy_use_coupon'] = 1
t7 = t7.groupby('User_id').agg('sum').reset_index()

t8 = user1[user1.Date!='null'][['User_id']]
t8['buy_total'] = 1
t8 = t8.groupby('User_id').agg('sum').reset_index()

t9 = user1[user1.Coupon_id!='null'][['User_id']]
t9['coupon_received'] = 1
t9 = t9.groupby('User_id').agg('sum').reset_index()

t10 = user1[(user1.Date_received!='null')&(user1.Date!='null')][['User_id','Date_received','Date']]
t10['user_date_datereceived_gap'] = t10.Date + ':' + t10.Date_received
t10.user_date_datereceived_gap = t10.user_date_datereceived_gap.apply(get_user_date_datereceived_gap)
t10 = t10[['User_id','user_date_datereceived_gap']]

t11 = t10.groupby('User_id').agg('mean').reset_index()
t11.rename(columns={'user_date_datereceived_gap':'avg_user_date_datereceived_gap'},inplace=True)
t12 = t10.groupby('User_id').agg('min').reset_index()
t12.rename(columns={'user_date_datereceived_gap':'min_user_date_datereceived_gap'},inplace=True)
t13 = t10.groupby('User_id').agg('max').reset_index()
t13.rename(columns={'user_date_datereceived_gap':'max_user_date_datereceived_gap'},inplace=True)

user1_feature = pd.merge(t,t1,on='User_id',how='left')
user1_feature = pd.merge(user1_feature,t3,on='User_id',how='left')
user1_feature = pd.merge(user1_feature,t4,on='User_id',how='left')
user1_feature = pd.merge(user1_feature,t5,on='User_id',how='left')
user1_feature = pd.merge(user1_feature,t6,on='User_id',how='left')
user1_feature = pd.merge(user1_feature,t7,on='User_id',how='left')
user1_feature = pd.merge(user1_feature,t8,on='User_id',how='left')
user1_feature = pd.merge(user1_feature,t9,on='User_id',how='left')
user1_feature = pd.merge(user1_feature,t11,on='User_id',how='left')
user1_feature = pd.merge(user1_feature,t12,on='User_id',how='left')
user1_feature = pd.merge(user1_feature,t13,on='User_id',how='left')
user1_feature.count_merchant = user1_feature.count_merchant.replace(np.nan,0)
user1_feature.buy_use_coupon = user1_feature.buy_use_coupon.replace(np.nan,0)
user1_feature['buy_use_coupon_rate'] = user1_feature.buy_use_coupon.astype('float') / user1_feature.buy_total.astype('float')
user1_feature['user_coupon_transfer_rate'] = user1_feature.buy_use_coupon.astype('float') / user1_feature.coupon_received.astype('float')
user1_feature.buy_total = user1_feature.buy_total.replace(np.nan,0)
user1_feature.coupon_received = user1_feature.coupon_received.replace(np.nan,0)
user1_feature.to_csv('data/user1_feature.csv',index=None)
user1_feature.head()

,User_id,count_merchant,user_min_distance,user_max_distance,user_mean_distance,user_median_distance,buy_use_coupon,buy_total,coupon_received,avg_user_date_datereceived_gap,min_user_date_datereceived_gap,max_user_date_datereceived_gap,buy_use_coupon_rate,user_coupon_transfer_rate
0,1439408,1.0,NaN,NaN,NaN,NaN,0.0,1.0,2.0,NaN,NaN,NaN,0.0,0.0
1,2029232,0.0,NaN,NaN,NaN,NaN,0.0,0.0,1.0,NaN,NaN,NaN,NaN,0.0
2,2223968,0.0,NaN,NaN,NaN,NaN,0.0,0.0,1.0,NaN,NaN,NaN,NaN,0.0
3,73611,0.0,NaN,NaN,NaN,NaN,0.0,0.0,1.0,NaN,NaN,NaN,NaN,0.0
4,3273056,0.0,NaN,NaN,NaN,NaN,0.0,0.0,1.0,NaN,NaN,NaN,NaN,0.0


## 5. user-merchant related features